---
## **📊 Flow**   
> step01 : Data   
> - 데이터 준비 및 분석   
> - 데이터 전처리
   #   
> setp02 : 모델 비교     
> - ResNet50, EfficientNet,VGG16
> - 모델 학습
> - 성능평가 : 정확도 재현율 f1
   #
> setp03 : 분석 실험   
> - Focal Loss, Class-balanced Loss 적용.   
> - Precision-Recall Trade-off   
> - XAI 기법   
> - 오분류 분석   

---
> ### step01 : Data   
> - 데이터 준비 및 분석   
> - 데이터 전처리

In [ ]:
from dataprocessing import *

# 데이터 경로
data_path = '/home/dibaeck/sketch/study_Data4Quality/task02_CovidClassifier/COVID19_1K'

---
>> 데이터 사이즈 다름 --> TASK : 데이터 리사이즈  

In [ ]:
check_image_sizes(data_path)

---
>> 클래스 불균형 : COVID19 데이터가 적음.  --> TASK : Data Augmentation   

In [ ]:
count_and_visualize_images(data_path)

클래스 불균형 해결 전략 : WeightedRandomSampler + FocalLoss

| 전략               | 설명                                                                 |
|--------------------|----------------------------------------------------------------------|
| **WeightedRandomSampler**          | 수 클래스를 더 자주 뽑히게 하는 샘플링 방식                |
| **Focal Loss**     | 소수 클래스의 어려운 샘플에 더 집중하는 loss function(의료 이미지에서 성능 개선에 효과적)                               |

---

In [1]:
######################################## setting 
from dataprocessing import *
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler

# 데이터 경로
data_path = '/home/dibaeck/sketch/study_Data4Quality/task02_CovidClassifier/COVID19_1K'

# 데이터 정규화 및 리사이즈, 텐서화
transform = get_transform()

no1_target_classes = ['PNEUMONIA', 'NORMAL']
train_dataset1, val_dataset1, test_dataset1 = get_customdatasets(data_path, transform,no1_target_classes)
train_loader1,val_loader1,test_loader1 = get_dataloaders(train_dataset1,val_dataset1,test_dataset1,batch_size=32)

no2_target_classes = ['COVID19', 'NORMAL']
train_dataset2, val_dataset2, test_dataset2 = get_customdatasets(data_path, transform,no2_target_classes)
train_loader2,val_loader2,test_loader2 = get_dataloaders(train_dataset2,val_dataset2,test_dataset2,batch_size=32)

no3_target_classes = ['NORMAL',"PNEUMONIA",'COVID19']
train_dataset3, val_dataset3, test_dataset3 = get_customdatasets(data_path, transform,no3_target_classes)
train_loader3,val_loader3,test_loader3 = get_dataloaders(train_dataset2,val_dataset2,test_dataset2,batch_size=32)

In [2]:
from models import *
import torch.optim as optim

# 모델 불러오기
# Binary Classification (PNEUMONIA vs NORMAL)
binary_model_pneumonia_vs_normal = CustomDenseNet(num_classes=2)

# Binary Classification (COVID19 vs NORMAL)
binary_model_covid19_vs_normal = CustomDenseNet(num_classes=2)

# Multi-class Classification (COVID19 vs PNEUMONIA vs NORMAL)
multi_class_model = CustomDenseNet(num_classes=3)


# 지표 설정
# 각각의 모델에 대해 이진 분류에서는 Binary Cross Entropy를, 다중 클래스 분류에서는 Cross Entropy를 사용
# 이진 분류 모델 (PNEUMONIA vs NORMAL, COVID19 vs NORMAL) 
criterion_binary = nn.BCEWithLogitsLoss()  # 이진 분류에서 사용
optimizer_binary = optim.Adam(binary_model_pneumonia_vs_normal.parameters(), lr=0.001)

# 다중 클래스 분류 모델 (COVID19 vs PNEUMONIA vs NORMAL)
criterion_multi_class = nn.CrossEntropyLoss()  # 다중 클래스 분류에서 사용
optimizer_multi_class = optim.Adam(multi_class_model.parameters(), lr=0.001)

/home/dibaeck/sketch/anaconda3/envs/dibk311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dibaeck/sketch/anaconda3/envs/dibk311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/dibaeck/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 56.8MB/s]


In [3]:
# 모델 학습
binary_model_pneumonia_vs_normal.model_train(train_loader1, criterion_binary, optimizer_binary)

torch.Size([32, 3, 224, 224])


RuntimeError: GET was unable to find an engine to execute this computation

In [ ]:
binary_model_covid19_vs_normal.model_train(train_loader2, criterion_binary, optimizer_binary)

multi_class_model.model_train(train_loader3,criterion_multi_class,optimizer_multi_class)
뭐가 문제인데ㅜㅠㅜㅠㅜㅠㅜ


# 모델 평가
binary_model_pneumonia_vs_normal.model_eval(test_loader)
binary_model_covid19_vs_normal.model_eval(test_loader)
multi_class_model.model_eval(test_loader)

In [ ]:
# 모델 훈련 및 평가 실행
train_and_evaluate_models(train_loader, test_loader)

### 모델 성능 비교

| **모델**                          | **정확도 (Accuracy)** | **정밀도 (Precision)** | **재현율 (Recall)** | **F1-Score** | **Confusion Matrix**                                |
|-----------------------------------|----------------------|------------------------|---------------------|--------------|-----------------------------------------------------|
| **PNEUMONIA vs NORMAL**           | 0.9655               | 0.9286                 | 0.6842              | 0.7879       | [[183   1] <br> [  6  13]]                         |
| **COVID19 vs NORMAL**             | 0.9360               | 0.9744                 | 0.7600              | 0.8539       | [[152   1] <br> [ 12  38]]                         |
| **Multi-class (COVID19 vs PNEUMONIA vs NORMAL)** | 0.9064 | 0.9064 | 0.9064 | 0.9064 | [[ 12   1   6] <br> [  1  39  10] <br> [  1   0 133]] |

---

### 결론

- **PNEUMONIA vs NORMAL 모델**: 높은 정확도와 정밀도, 하지만 재현율이 상대적으로 낮습니다.
- **COVID19 vs NORMAL 모델**: 높은 정확도, 정밀도, 재현율을 보이며, 비교적 균형 잡힌 성능을 보여줍니다.
- **Multi-class 모델**: 세 가지 클래스에 대해 고른 성능을 보여주며, F1-Score와 정확도가 90% 이상으로 우수합니다.

**최종 평가**:
- **COVID19 vs NORMAL 모델**은 정밀도와 재현율이 균형을 이루어 가장 우수한 성능을 보입니다.
- **Multi-class 모델**은 세 가지 클래스를 잘 분리하며 좋은 성능을 보였습니다.

---

1]Multi-class분류에서 사용된 loss 가 모든 클래스에 적절했는지 분석하고,Focal loss,class-balancedloss등을 적용한 실험을 수행해보세요.

1) Loss 함수 분석 및 Focal Loss, Class-Balanced Loss 실험   
Multi-class 분류에서 기본적으로 사용되는 loss 함수는 Cross-Entropy Loss입니다. 이를 Focal Loss나 Class-Balanced Loss로 변경하여 성능을 비교할 수 있습니다.
   
Focal Loss:   
Focal Loss는 클래스 불균형 문제를 해결하기 위한 loss 함수입니다. Cross-Entropy Loss는 클래스 간 불균형이 심할 때 성능이 떨어질 수 있는데, Focal Loss는 어려운 샘플에 가중치를 주어 성능을 개선할 수 있습니다.

In [ ]:
import torch
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25, num_classes=3):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.num_classes = num_classes
        self.ce_loss = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce_loss(inputs, targets)
        pt = torch.exp(-ce_loss)  # For each class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()
    
class ClassBalancedLoss(nn.Module):
    def __init__(self, class_weights):
        super(ClassBalancedLoss, self).__init__()
        self.class_weights = class_weights
        self.ce_loss = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, inputs, targets):
        return self.ce_loss(inputs, targets)
    
from sklearn.metrics import precision_score, recall_score

def adjust_threshold(predictions, targets, threshold=0.5):
    pred_classes = (predictions[:, 1] > threshold).float()  # COVID-19 클래스의 확률을 기준으로 예측
    precision = precision_score(targets, pred_classes, average='binary', pos_label=1)
    recall = recall_score(targets, pred_classes, average='binary', pos_label=1)
    return precision, recall

import cv2
import numpy as np
import torch
from torchvision import models

class GradCAM:
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.gradients = None
        self.activations = None

    def save_gradient(self, grad):
        self.gradients = grad

    def forward(self, x):
        activations = self.model.features(x)
        activations.register_hook(self.save_gradient)
        return activations

    def generate_gradcam(self, input_image, target_class):
        activations = self.forward(input_image)
        self.model.zero_grad()
        activations[target_class].backward(retain_graph=True)
        gradients = self.gradients
        pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
        weighted_activations = activations * pooled_gradients.view(1, -1, 1, 1)
        gradcam = torch.mean(weighted_activations, dim=1).squeeze()
        gradcam = np.maximum(gradcam.cpu().detach().numpy(), 0)
        gradcam = cv2.resize(gradcam, (input_image.size(2), input_image.size(3)))
        return gradcam

import shap
import numpy as np

# SHAP을 사용하여 모델 해석
def explain_with_shap(model, dataloader):
    explainer = shap.KernelExplainer(model.predict, data=dataloader)
    shap_values = explainer.shap_values(dataloader)
    shap.summary_plot(shap_values, dataloader)

from sklearn.metrics import confusion_matrix

def analyze_confusion_matrix(predictions, targets):
    cm = confusion_matrix(targets, predictions)
    print("Confusion Matrix:\n", cm)

# 예시: 오분류된 샘플에 대해 가중치를 더 주기
def reweight_loss_function(model, predictions, targets):
    # 잘못 분류된 샘플에 대해 가중치 증가
    incorrect_samples = predictions != targets
    weights = torch.where(incorrect_samples, 2.0, 1.0)
    loss = F.cross_entropy(predictions, targets, weight=weights)
    return loss


In [ ]:
# 모델 학습 시 사용될 loss 함수 설정
criterion = FocalLoss(gamma=2, alpha=0.25, num_classes=3)  # 예시로 FocalLoss 사용

# 학습 과정에서 해당 loss를 사용하여 모델 훈련


In [ ]:
thresholds = [0.3, 0.5, 0.7]  # 여러 threshold 값 시도
for threshold in thresholds:
    precision, recall = adjust_threshold(predictions, targets, threshold)
    print(f"Threshold: {threshold} -> Precision: {precision}, Recall: {recall}")


In [ ]:
gradcam = GradCAM(model)
cam_output = gradcam.generate_gradcam(input_image, target_class)
# 결과를 시각화
plt.imshow(cam_output, cmap='jet')
plt.show()
